In [1]:
from datasets import load_from_disk, Dataset
import torch

/home/elena/miniconda/envs/emcomm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
original_ds = load_from_disk("../../../datasets/coco_val_images")

In [3]:
original_ds

Dataset({
    features: ['coco_url', 'captions', 'image'],
    num_rows: 5000
})

In [4]:
print(torch.cuda.device_count())
print([torch.cuda.get_device_name(i) for i in range(torch.cuda.device_count())])

2
['NVIDIA TITAN RTX', 'NVIDIA TITAN RTX']


In [11]:
from torchvision import models, transforms

# Load ResNet model and set to eval mode
resnet = models.resnet152(pretrained=True)
resnet.eval()

# Define image preprocessing
preprocess = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

def image_to_features(example):
    # Convert PIL image to tensor and preprocess
    image = example["image"]
    input_tensor = preprocess(image).unsqueeze(0).to("cuda" if torch.cuda.is_available() else "cpu")
    resnet.to("cuda" if torch.cuda.is_available() else "cpu")
    with torch.no_grad():
        features = resnet(input_tensor)
    return {"image_tensor": features.cpu().squeeze().numpy()}

/home/elena/miniconda/envs/emcomm/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/elena/miniconda/envs/emcomm/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /home/elena/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


100%|██████████| 230M/230M [00:04<00:00, 51.6MB/s] 


In [ ]:
# import os

# print(os.cpu_count())

In [12]:
new_ds = original_ds.map(
    lambda example: {
        "captions": example["captions"],
        "features": torch.tensor(image_to_features(example)["image_tensor"])
    },
    remove_columns=[col for col in original_ds.column_names if col not in ["captions"]],
    # num_proc=24
)

Map: 100%|██████████| 5000/5000 [01:59<00:00, 41.71 examples/s]


In [13]:
len(new_ds['features'][0])

1000

In [14]:
new_ds.save_to_disk("../../../datasets/coco_val_features_resnet_152")

Saving the dataset (1/1 shards): 100%|██████████| 5000/5000 [00:00<00:00, 12772.05 examples/s]


In [15]:
len(new_ds[0]['features'])

1000

In [ ]:
ds = load_from_disk("../../../datasets/coco_val_features")
ds

In [ ]:
ds[0]